In [ ]:
%%capture
import os
import pandas as pd
import numpy as np
from dj_notebook import activate
from pathlib import Path

env_file = os.environ["INTECOMM_ENV"]
analysis_folder = Path(os.environ["INTECOMM_ANALYSIS_FOLDER"])
reports_folder = Path(os.environ["INTECOMM_ANALYSIS_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from intecomm_analytics.dataframes import get_df_main_1858
from edc_constants.constants import NO, YES
from intecomm_analytics.constants import HIV_ALONE, HTN_ALONE, DM_ALONE, HTN_DM


In [ ]:
df_main = get_df_main_1858(None)


In [ ]:
[col for col in df_main.columns if col.startswith("primary")]

In [ ]:
df_main[[col for col in df_main.columns if col.startswith("primary")]]


In [ ]:
def get_yes_no(s):
    if pd.notna(s.primary_controlled):
        return YES if s.primary_controlled == 1 else NO
    return 'Missing'

In [ ]:
df = df_main[df_main.primary_cohort.isin([HTN_ALONE, DM_ALONE, HTN_DM])].groupby(by=["assignment", "primary_controlled"], dropna=False).size().reset_index(name="count")
df["primary_controlled"] = df.apply(get_yes_no, axis=1)
df_pivot = df.pivot_table(index="primary_controlled", columns="assignment", values="count", fill_value=0).reset_index()
df_pivot.columns.name = None
df_pivot['total'] = df_pivot['a'] + df_pivot['b']
totals = df_pivot.sum(numeric_only=True)
totals['primary_controlled'] = 'Total'
df_pivot = pd.concat([df_pivot, totals.to_frame().T], ignore_index=True)
df_pivot = pd.concat([df_pivot.loc[0:2].sort_values(by="primary_controlled", ascending=False), df_pivot.loc[3:]]).reset_index(drop=True)
df_pivot


In [ ]:
df = df_main[df_main.primary_cohort.isin([HIV_ALONE])].groupby(by=["assignment", "primary_controlled"], dropna=False).size().reset_index(name="count")
df["primary_controlled"] = df.apply(get_yes_no, axis=1)
df_pivot = df.pivot_table(index="primary_controlled", columns="assignment", values="count", fill_value=0).reset_index()
df_pivot.columns.name = None
df_pivot['total'] = df_pivot['a'] + df_pivot['b']
totals = df_pivot.sum(numeric_only=True)
totals['primary_controlled'] = 'Total'
df_pivot = pd.concat([df_pivot, totals.to_frame().T], ignore_index=True)
df_pivot = pd.concat([df_pivot.loc[0:1].sort_values(by="primary_controlled", ascending=False), df_pivot.loc[2:]]).reset_index(drop=True)
df_pivot